# Reddit API scraping with Python

In [1]:
# General libs
import math
import json
import time

# Connection
import requests

# Data
import pandas as pd

### Connecting to Reddit

In [2]:
def getHeaders():
    with open("secret.json") as file:
        client_id, secret, password = json.load(file).values()
    
    user_data = {
        "grant_type":"password",
        "username":"Hungry_Fuel_2913",
        "password":password
    }
    
    auth = requests.auth.HTTPBasicAuth(client_id, secret)
    headers = {"User-Agent":"Python-API/0.0.1"}
    res = requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=user_data, headers = headers)
    TOKEN = res.json()["access_token"]
    headers["Authorization"] = f"bearer {TOKEN}"
    
    return headers

In [3]:
headers = getHeaders()

### Getting my User Info

In [4]:
my_info = requests.get("https://oauth.reddit.com/api/v1/me", headers=headers).json()

In [5]:
print(my_info.keys())

dict_keys(['is_employee', 'seen_layout_switch', 'has_visited_new_profile', 'pref_no_profanity', 'has_external_account', 'pref_geopopular', 'seen_redesign_modal', 'pref_show_trending', 'subreddit', 'pref_show_presence', 'snoovatar_img', 'snoovatar_size', 'gold_expiration', 'has_gold_subscription', 'is_sponsor', 'num_friends', 'features', 'can_edit_name', 'verified', 'new_modmail_exists', 'pref_autoplay', 'coins', 'has_paypal_subscription', 'has_subscribed_to_premium', 'id', 'has_stripe_subscription', 'oauth_client_id', 'can_create_subreddit', 'over_18', 'is_gold', 'is_mod', 'awarder_karma', 'suspension_expiration_utc', 'has_verified_email', 'is_suspended', 'pref_video_autoplay', 'has_android_subscription', 'in_redesign_beta', 'icon_img', 'has_mod_mail', 'pref_nightmode', 'awardee_karma', 'hide_from_robots', 'password_set', 'link_karma', 'force_password_reset', 'total_karma', 'seen_give_award_tooltip', 'inbox_count', 'seen_premium_adblock_modal', 'pref_top_karma_subreddits', 'has_mail', 

## Scrapping Reddit Argentina

### Hot topics

If we use the endpoint /subreddit/hot we get the hottest topics of that subreddit. 

In [6]:
headers = getHeaders()
res = requests.get("https://oauth.reddit.com/r/argentina/hot", headers=headers)

In [7]:
def extractPostInfo(keys_to_extract=[], postList=[]):
    df = pd.DataFrame()

    for post in postList:
        data = post["data"]
        row = {}
        row["kind"] = post["kind"]
        for key in keys_to_extract:
            row[key] = data[key]
        df = df.append(row, ignore_index=True)

    return df

In [8]:
print(res.json()["data"]["children"][0])

{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'argentina', 'selftext': 'Thread Diario de Dudas y Consultas!\n\nEntra a nuestro [Discord](https://discord.gg/PeaSPMPn46) y charla con la comunidad!', 'author_fullname': 't2_f0mqjq9a', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Thread Diario de Dudas, Consultas y Mitaps - 23/03', 'link_flair_richtext': [{'e': 'text', 't': 'Sticky 📌'}], 'subreddit_name_prefixed': 'r/argentina', 'hidden': False, 'pwls': 6, 'link_flair_css_class': 'sticky', 'downs': 0, 'thumbnail_height': None, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_tkpp33', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 0.95, 'author_flair_background_color': None, 'subreddit_type': 'public', 'ups': 17, 'total_awards_received': 0, 'media_embed': {}, 'thumbnail_width': None, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': [], 'secure_media': None, 'is_reddit_media_d

In [9]:
keys_to_extract = ["subreddit", "title", "author", 
                   "permalink", "url", "num_comments", "ups", "selftext", "id",
                   "downs", "upvote_ratio", "score","created", "edited", "author_fullname", "subreddit_id" ]

df = extractPostInfo(keys_to_extract, res.json()["data"]["children"])

In [10]:
df.sort_values(["upvote_ratio","score"], ascending=True)[["title","upvote_ratio","ups", "downs", "score"]].head()

,title,upvote_ratio,ups,downs,score
1,Miercoles de Rant,0.81,13.0,0.0,13.0
21,Se puede,0.84,59.0,0.0,59.0
7,Hace un tiempo me regalaron una tablet de dibu...,0.84,138.0,0.0,138.0
26,Javier Milei aseguró que la solución contra la...,0.87,127.0,0.0,127.0
18,"""Por favor no me pese"": la campaña para evitar...",0.90,87.0,0.0,87.0


It is interesting to note that, even though we observe many upvote ratios values under 1, the number of downs is always zero. Maybe it is a bug, or maybe Reddit doesn't want us to know the number of downvotes a post has. Whatever the case, we can infer the number of downvotes from the number of upvotes an the ratio

In [11]:
def guessDownsFromRatio(ups, ratio):
    ups = float(ups)
    ratio = float(ratio)
    if ratio > 0:
        return math.floor(ups*( (1-ratio) / ratio  ) )
    else:
        return None
    
df["guessed_downs"] = df.apply(lambda row: guessDownsFromRatio(row["ups"], row["upvote_ratio"]), axis=1)

In [12]:
df.sort_values(["upvote_ratio","score"], ascending=True)[["title","upvote_ratio","ups", "guessed_downs", "score"]].head()

,title,upvote_ratio,ups,guessed_downs,score
1,Miercoles de Rant,0.81,13.0,3,13.0
21,Se puede,0.84,59.0,11,59.0
7,Hace un tiempo me regalaron una tablet de dibu...,0.84,138.0,26,138.0
26,Javier Milei aseguró que la solución contra la...,0.87,127.0,18,127.0
18,"""Por favor no me pese"": la campaña para evitar...",0.90,87.0,9,87.0


In [13]:
df.iloc[18]["permalink"]

'/r/argentina/comments/tl4c8k/por_favor_no_me_pese_la_campaña_para_evitar_que/'

## All Posts

In [14]:
def getAllPosts(keys_to_extract, headers, subreddit="argentina", paused=True):
    '''
    This function accesses a subredit and then iteratively retrieves the last
    100 posts until there are no more posts to be accessed, then extracts the
    desired data from the json responses and returns it as a pandas dataframe.
    
    This function recieves a list of the keys of the json response that are 
    to be extracted    as an argument. The headers allow the function to get 
    access to the reddit api.
    
    The default subreddit is /argentina since this function was originaly created
    to scrap that subreddit. Another subreddit can be passed as an argument.
    The script takes a pause of 5 seconds between requests to avoid making too much
    calls to the api. This can be overriden by pasing the pause=False argument.
    

    '''

    print(f"Obtaining data from Reddit/{subreddit}...")
    
    # Create an empty dataframe and get the first group of posts
    df = pd.DataFrame()
    res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/new", headers=headers, params={"limit":"100"})
    # If something went wrong, stop.
    if(res.status_code != 200):
        print("Data cannot be accessed", f"Status code: {res.status_code}")
        return df

    # Keep getting older posts
    while (res.status_code == 200):
        # Extact current data and store it in the dataframe
        df_current = extractPostInfo(keys_to_extract, res.json()["data"]["children"])
        if df_current.shape[0] == 0:
            break
        df = pd.concat([df, df_current])
        # Get the id of the last obtained post to know where to start next
        last = df.iloc[-1]["kind"] + "_" + df.iloc[-1]["id"]
        print("last:", last)
        # This delay is there just in case, we don't want to get banned
        if paused:
            time.sleep(5) 
        # Show some feedback on screen to let the user know what is going on
        print(f"obtained {df_current.shape[0]} posts")
        print(f"total: {df.shape[0]} posts")
        print("next!")
        # Try to get the next group of posts and restart the loop
        res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/new", headers=headers, params={"limit":"100", "after":last})
        print("Status code", res.status_code)
        
    print("done!")
    # We now guess the number of downvotes:
    df["guessed_downs"] = df.apply(lambda row: guessDownsFromRatio(row["ups"], row["upvote_ratio"]), axis=1)
    # Return the data
    return df      

## Expand and unify data

Previously we have extracted all the available posts in a particular subreddit, and stored that as a csv, named with the timestamp at the moment of the extraction. The information that a particular post contains changes over time. New versions of the csv will contain both new posts and newer version of some of the posts in previous csv. 
<br>
What we need to do is:
<ol>
    <li>Load the current dataframe and the previous csv </li>
    <li>New posts are added to the old dataset</li>
    <li>Existing posts are updated with the newest data</li>
    <li>Posts not present in current dataframe are left unchanged</li>
</ol>

In [15]:
def combineOnColAndUpdate(old_df, new_df, index_col):
    '''
    Takes a dataframe with the existing data and
    updates the threads that exist in both dataframes
    with values from the latest version.
    It also adds the new threads that were not present in
    the old dataframe.
    Threads are identified by their fullname, wich is
    composed of the kind and the id separated by a lodash

    '''
    a = new_df.set_index(index_col)
    b = old_df.set_index(index_col)
    df = (a.combine_first(b)).reset_index().reindex(columns=old_df.columns)
    return df

### Getting all post and updating our database

In [16]:
#Old data
save_file = "scrapped_data.csv"
old_df = pd.read_csv(save_file)

# Scrap new data
headers = getHeaders()
new_df = getAllPosts(keys_to_extract, headers, subreddit="argentina")

# Update data
df = combineOnColAndUpdate(old_df, new_df, "id")


df.info()

Obtaining data from Reddit/argentina...
last: t3_tkdphm
obtained 100 posts
total: 100 posts
next!
Status code 200
last: t3_tjdvei
obtained 100 posts
total: 200 posts
next!
Status code 200
last: t3_ti2uby
obtained 100 posts
total: 300 posts
next!
Status code 200
last: t3_thd3xt
obtained 100 posts
total: 400 posts
next!
Status code 200
last: t3_tgpdey
obtained 100 posts
total: 500 posts
next!
Status code 200
last: t3_tg20uz
obtained 100 posts
total: 600 posts
next!
Status code 200
last: t3_tf54bq
obtained 100 posts
total: 700 posts
next!
Status code 200
last: t3_teax8m
obtained 100 posts
total: 800 posts
next!
Status code 200
last: t3_tde4pl
obtained 100 posts
total: 900 posts
next!
Status code 200
last: t3_tc4ztx
obtained 75 posts
total: 975 posts
next!
Status code 200
done!
<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 74
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kind          

In [17]:
new_df.to_csv(f"reddit-argentina-v{math.floor(time.time())}.csv",index=False)
df.to_csv(save_file,index=False)

In [21]:
# print(json.dumps(res.json()["data"]["children"][6], indent=4))

In selftext we have the post content, if the post has some text in it and is not just an image, video or link. <br>
I want to check that the text is saved correctly in selftext if such a text exists.

In [39]:
id = "tl0u39"

result = df[df["id"] == id]
print(result.iloc[0]["title"])
print("\nCreated:", result.iloc[0]["created"])
print("-----------------------\n")

print(result.iloc[0]["selftext"])

Perdonen chicos, tengan cuidado con los lugares de comida por peso. Me vino un gusano muerto de 1 cm en la comida.

Created: 1648054293.0
-----------------------

Resulta que compré comida ahi y me vino un gusano. No se preocupen, ya me metí los dedos por la garganta y estoy viendo si tengo que tomar algún antiparasitario.
Sé que no todos los locales del estilo son así necesariamente, pero tengan cuidado. La comida por lo general esta expuesta al aire y, por consiguiente, a las moscas que pueden dejar sus huevos en la comida y hacer que salgan gusanos. ¿Les pasó alguna vez? ¿Qué piensan al respecto?

Pido perdón porque fui el que recomendó ir a lugares de comida por peso en otro post XD.

Edit: Era el Wok Express de Rodríguez Peña entre Av Santa Fe y Arenales. Era un gusano de la verdura y al final no media 1 cm sino que medio cm.


In [ ]:
df